<a href="https://colab.research.google.com/github/popelucha/digital-humanities/blob/main/Naive_Bayes_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Naive Bayes Classifier
Zkusíme si úplně jednoduchý klasifikátor, který bude rozhodovat, zda jsou začátky článků z Wikipedie úvodem ke článku o člověku, o místě, nebo o uměleckém díle.

Použijeme NL ToolKit (NLTK).

Napřed se podíváme na data.

In [ ]:
import nltk
import random
from nltk.tokenize import RegexpTokenizer


In [ ]:
!wget http://aurora.fi.muni.cz/~xpopelk/artwork.txt
!wget http://aurora.fi.muni.cz/~xpopelk/human.txt
!wget http://aurora.fi.muni.cz/~xpopelk/town.txt

--2024-10-22 08:50:58--  http://aurora.fi.muni.cz/~xpopelk/artwork.txt
Resolving aurora.fi.muni.cz (aurora.fi.muni.cz)... 147.251.51.10
Connecting to aurora.fi.muni.cz (aurora.fi.muni.cz)|147.251.51.10|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://aurora.fi.muni.cz/~xpopelk/artwork.txt [following]
--2024-10-22 08:50:59--  https://aurora.fi.muni.cz/~xpopelk/artwork.txt
Connecting to aurora.fi.muni.cz (aurora.fi.muni.cz)|147.251.51.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28032 (27K) [text/plain]
Saving to: ‘artwork.txt’

artwork.txt         100%[===================>]  27.38K   126KB/s    in 0.2s    

2024-10-22 08:51:00 (126 KB/s) - ‘artwork.txt’ saved [28032/28032]

--2024-10-22 08:51:00--  http://aurora.fi.muni.cz/~xpopelk/human.txt
Resolving aurora.fi.muni.cz (aurora.fi.muni.cz)... 147.251.51.10
Connecting to aurora.fi.muni.cz (aurora.fi.muni.cz)|147.251.51.10|:80... connected.
HTTP request sent, 

In [ ]:
def get_data(filenames, rows = 0):
  labeled_text = []
  for filename in filenames:
      i = 0
      with open(filename, encoding='utf-8') as f:
         for line in f:
            labeled_text.append((line.strip(), filename.split(".")[0]))
            i += 1
            if rows!=0 and i>rows:
               break
  return labeled_text

dataset = ["human.txt","artwork.txt", "town.txt"]
#dataset = ["artwork.txt", "town.txt"]

print("Loading data from {}".format(dataset))
labeled_texts = get_data(dataset)

Loading data from ['human.txt', 'artwork.txt', 'town.txt']


In [ ]:
"artwork.txt".split(".")

['artwork', 'txt']

In [ ]:
"artwork.txt".split(".")[0]

'artwork'

In [ ]:
"artwork.txt".split(".")[1]

'txt'

In [ ]:
print(len(labeled_texts), "labeled texts")
labeled_texts[421]

1056 labeled texts


('RRRrrrr!!! je francouzský komediální film , který v roce 2004 natočil režisér Alain',
 'artwork')

# Features (Atributy)
Jako jediný atribut pro rozpoznávání vezmeme slovo. Každý text je tedy reprezentován jako slovník slov.

In [ ]:
def classify(text):
  f = word_feature(all_words, text)
  text_class = classifier.classify(f)
  dist = classifier.prob_classify(f)
  desc = ""
  for label in dist.samples():
      desc += " "+("%s: %f" % (label, dist.prob(label)))
  return text_class, desc.strip()

def word_feature(all_words, text):
  z = {}
  for word in all_words:
      z[word] = word in tokenizer.tokenize(text)
  return z

In [ ]:
tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')
random.shuffle(labeled_texts)

all_words = set(word for passage in labeled_texts for word in tokenizer.tokenize(passage[0]))
print("all_words", len(all_words))
split_train = int(len(labeled_texts)*.75)
print("train samples", split_train)
train_set = [(word_feature(all_words, x[0]), x[1]) for x in labeled_texts[:split_train]]
test_set = [(word_feature(all_words, x[0]), x[1]) for x in labeled_texts[split_train:]]

print(len(train_set), len(test_set))

all_words 3721
train samples 792
792 264


In [ ]:
import pprint
print("Training sample", labeled_texts[421])
pprint.pprint(train_set[421])

Training sample ('Joseph Lee Henry je americký zpěvák , kytarista, hudební producent a skladatel', 'human')
({'!': False,
  '!!!': False,
  '"': False,
  '",': False,
  '".': False,
  '"1990"': False,
  '"Chatrč"': False,
  '"Churchill': False,
  '"Freak': False,
  '"Kanibalové': False,
  '"Letopisy': False,
  '"Main': False,
  '"Otto': False,
  '"Passfield': False,
  '"Tintinova': False,
  '"Vinnetou': False,
  '"v': False,
  '"Šmu\'el': False,
  '&amp;amp;': False,
  "'Rourkea": False,
  "'animation": False,
  '(*': False,
  '(14.': False,
  '(Ph': False,
  '(Ph.D.': False,
  '(bas,': False,
  '(rodiče': False,
  ')': False,
  '),': False,
  '*': False,
  ',': True,
  ',5': False,
  '-': False,
  '-1936': False,
  '-1972': False,
  '-1978': False,
  '-Aškur': False,
  '-Buriaš': False,
  '-Dadžani': False,
  '-FORCE': False,
  '-Francie': False,
  '-Grid': False,
  '-Horákov': False,
  '-Iluna': False,
  '-James': False,
  '-Kabít-Achchéšu': False,
  '-Korouhvice': False,
  '-Louise'

In [ ]:
len(train_set[421][0])

3721

In [ ]:
print("Start training")
%time classifier = nltk.NaiveBayesClassifier.train(train_set)

print("test set accuracy: {}".format(nltk.classify.accuracy(classifier, test_set)))
print("train set accuracy: {}".format(nltk.classify.accuracy(classifier, train_set)))


Start training
CPU times: user 3.61 s, sys: 19.6 ms, total: 3.63 s
Wall time: 3.62 s
test set accuracy: 0.9962121212121212
train set accuracy: 1.0


In [ ]:
classifier.show_most_informative_features(35)

Most Informative Features
                       z = True           artwor : town   =     86.6 : 1.0
                    roku = True           artwor : town   =     79.7 : 1.0
                    jsou = True             town : human  =     55.7 : 1.0
                     byl = True            human : artwor =     21.4 : 1.0
                  bývalý = True            human : town   =     21.2 : 1.0
                     asi = True             town : artwor =     20.9 : 1.0
                   Karel = True            human : artwor =      8.1 : 1.0
                      je = False           human : artwor =      7.7 : 1.0
                       3 = True             town : artwor =      7.4 : 1.0
                       a = True            human : town   =      7.2 : 1.0
                     pro = True           artwor : human  =      7.2 : 1.0
                    Brno = True             town : human  =      6.9 : 1.0
                       6 = True             town : human  =      6.3 : 1.0

In [ ]:
test_sentence = u'Patrick George "Pat" Troughton byl anglický herec'
print(test_sentence)
print(test_sentence in [t for t, c in labeled_texts])
print(classify(test_sentence))
test_sentence = u'Dune 2000 je vylepšená verze hry Dune II , vydanou společností Virgin/EA pod hlavičkou firmy Westwood Studios .'
print(test_sentence)
print(test_sentence in [t for t, c in labeled_texts])
print(classify(test_sentence))
test_sentence = u'William Shakespeare (pokřtěn 26. dubna 1564 - zemřel 23. dubna 1616) byl významný anglický básník a dramatik'
print(test_sentence)
print(test_sentence in [t for t, c in labeled_texts])
print(classify(test_sentence))
test_sentence = u'Pleurón (starořecky Πλευρών) je v řecké mytologii syn aitólskeho krále Aitola a jeho manželky Pronoé.[1]'
#test_sentence = u'Jim Pons (* 14. března 1943, Santa Monica, Kalifornie, USA[1]) je americký baskytarista a zpěvák'
print(test_sentence)
print(test_sentence in [t for t, c in labeled_texts])
print(classify(test_sentence))


Patrick George "Pat" Troughton byl anglický herec
False
('human', 'town: 0.000000 human: 0.999987 artwork: 0.000013')
Dune 2000 je vylepšená verze hry Dune II , vydanou společností Virgin/EA pod hlavičkou firmy Westwood Studios .
False
('human', 'town: 0.000036 human: 0.979329 artwork: 0.020636')
William Shakespeare (pokřtěn 26. dubna 1564 - zemřel 23. dubna 1616) byl významný anglický básník a dramatik
False
('human', 'town: 0.000000 human: 1.000000 artwork: 0.000000')
Pleurón (starořecky Πλευρών) je v řecké mytologii syn aitólskeho krále Aitola a jeho manželky Pronoé.[1]
False
('human', 'town: 0.000059 human: 0.999494 artwork: 0.000448')
